# Sudoku

This workbook contains a Sudoku generator which solves sudoku puzzles

## Specification

### Sudoku

A sudoku puzzle consists of a $9 \times 9$ __grid__, where each location in the grid is referred to as a __cell__. Each __cell__ can either be blank, meaning its value is unknown, or contain any of the digits 1 to 9, inclusive.

A sudoku puzzle is provided partially filled in, and you solve the puzzle by filling in all of the blank cells with one of the digits 1 to 9, subject to a set of __constraints__. The constraints are:
1. Each digit can only appear once in each row.
2. Each digit can only appear once in each column.
3. Each digit can only appear once in each square. The squares are all $3 \times 3$ groups of cells, laid out across the grid, so there are 3 across, 3 down, for 9 squares all in (see the below visualisation which shows the squares).

The only remaining rule is that there can only be one solution. However, to introduce some terminology, each of the size 9 groups of cells in which each of the digits can only appear once is referred to as a __peer group__. Corresponding to the three constraints there are hence three kinds of peer group: __row peer group__, __column peer group__ and __square peer group__. Note that each cell belongs to one of each.

If you want another explanation of Sudoku then there is Wikipedia: https://en.wikipedia.org/wiki/Sudoku
Here is one of the many online implementations, in case you have never played it before: https://sudoku.game/ (You can, however, waste a lot of time doing this. Best avoided!)

### Goal

To generate random Sudoku puzzles and print them out such that they look like this:
```
    3|  2  |6   
9    |3   5|    1
    1|8   6|4    
-----+-----+-----
    8|1   2|9    
7    |     |    8
    6|7   8|2    
-----+-----+-----
    2|6   9|5    
8    |2   3|    9
    5|  1  |3    
```
For each generated puzzle it must first print out the puzzle with gaps for the unknown values, as you would present it to a human to solve, followed by the solved puzzle, where every value is filled in.

## Implementation

A specification only defines what the code is to do. Any complex program contains hundreds of design decisions that are not explicitly specified and left up to the programmer to decide. So this section is not part of the specification, rather it is a high level overview of the particular implementation that has been chosen.

Generating puzzles is in general a very hard problem. However, for many puzzles there is a stupid approach: Generate random puzzles that may be invalid, and then check if they are valid or not. Keep generating random puzzles until one passes the validity check and return that as the generated puzzle. This is the approach taken here. The flaw with this approach is that you may have to sample millions of puzzles until one passes the validity test - you usually have to be at least a little clever about the generation. In the case of Sudoku it is the case that you need at least 17 cells filled in for there to be any chance of it having only one solution. So this solution (initially) fills in 17 cells of the grid. Instead of testing validity after filling in all 17 cells with random digits, which would be horrifically inefficient, it selects cells at random and fills them in with a random digit that does not violate any of the cells already filled in (using constraint propagation, as described below for the solver). Having generated a possible puzzle it tries to solve it. If it has one solution then it is declared valid and output. If it has more than one solution it adds another digit to a random cell and tries again. Sometimes it will jump to zero solutions however, at which point it declares failure and starts all over again, generating a new puzzle from scratch. In practise this approach will normally produce a valid puzzle in under 4 attempts.

The above approach requires a Sudoku solver. Solving a Sudoku is an example of a _constraint satisfaction problem_ (https://en.wikipedia.org/wiki/Constraint_satisfaction_problem), an extremely important kind of problem that is used for far more serious problems than silly puzzles. Sudokus are conveniently simple compared to more general cases, and can be solved by combining two techniques: Search (which is part of minimax, which was used for noughts and crosses) and constraint propagation. Constraint propagation is where changing a value results in every associated constraint being propagated across the state, to limit the search to valid states only. This is the approach taken by the below code. Note that 'easy' Sudokus can typically be solved without search, i.e. the constraint propagation alone causes the grid to be entirely filled in!

Constraint propagation is a fairly open ended procedure - in a sense it does all of the 'obvious' moves immediately following a move. The definition of 'obvious' is left up to the coder! However, there is a useful minimum set, used here. It is defined in the sense of only allowing valid states, such that we never have to check the three constraints given above as we can never enter a state where one of them is violated. In other words if any of the constraint propagation rules given below was removed we would have to start checking for the constraints given above, and the search space would explode, leaving the algorithm much too slow. For Sudoku a good minimum set consists of two rules:
1. If the value of a cell is known then remove that value from all other cells in its three peer groups.
2. If a value can only go in one place within a peer group put it there.

Finally, there is a question of how to represent the $9 \times 9$ grid. Instead of recording the value assigned to each cell we record a set of 9 flags (`bool`), one per digit, that indicate that the given digit is allowed in the indexed cell (`True`) or that it is not allowed (`False`). This means that a cell being assigned a specific value actually means that only one of its 9 flags is `True`. The two propagation rules above are hence coded in response to setting a specific digit for a call to `False`, to indicate it can never take that value.


## Code

The below code block is a complete Sudoku solver plus a generator that uses it

In [3]:
import numpy


class SudokuState:
    """This represents the state of a Sudoku grid, as a list of valid numbers for each
    cell in the 9x9 grid. Provides functions that do constraint propagation when you
    remove a number as valid from a cell, so the list remains consistant for all cells."""
    __slots__ = ['allowed'] # Fixes the variables this class can have - reduces memory consumption!
    
    # Helper variable - slices that convert any coordinate into
    # the range of the 3x3 square peer group (same for both rows and cols)...
    # (zero based)
    __square = (slice(0,3), slice(0,3), slice(0,3),
                slice(3,6), slice(3,6), slice(3,6),
                slice(6,9), slice(6,9), slice(6,9))
    
    def __init__(self):
        """Constructs a SudokuState where every number is allowed, that is no constraints
        have yet been added."""
        # Represents the state of a board by indicating which values are allowed
        # in each cell (True) and which are not (False). For instance, self.allowed[4,2]
        # contains the flags for row 4, column 2 of the grid (0 based), and self.allowed[4,2,5]
        # will be True if a '6' is allowed in that cell, False if it is not (note the offset by
        # 1 due to numpy arrays being zero based when Sudoku is one based).
        self.allowed = numpy.ones((9,9,9), dtype=bool)


    def copy(self):
        """Returns a copy of this state object."""
        ret = SudokuState()
        ret.allowed[:,:,:] = self.allowed
        return ret


    def load(self, s):
        """Given a string, of length 81, representing the cells of a Sudoku grid.
        The digits 1-9 do what you expect, provide the value of that cell, while
        the digit 0 is used to indicate a cell that is to be left empty (unknown).
        Given this representation this loads the puzzle into this state. Should
        only be called on a state that has not been modified after construction,
        as all this does is remove all values other than the one provided for
        known cells. Note that because this does constraint propogation while
        loading it will effectively start solving the puzzle, and in the case
        of easy puzzles exit with it solved!"""
        for row in range(9):
            for col in range(9):
                offset = row*9 + col
                if s[offset]!='0':
                    value = int(s[offset])
                    for val in range(1,10):
                        if val==value:
                            continue
                        if self.allowed[row, col, val-1]:
                            self.remove(row, col, val)

    def valid(self):
        """Returns True if the state is valid, False if not. False means
        that one of the cells has no remaining values that can be
        assigned to it, implying the constraints have been violated."""
        # Note that this code uses the fact that if you sum over a boolean
        # array numpy interprets True to equal 1 and False to equal 0...
        return not numpy.any(self.allowed.sum(axis=2)==0)
    
    
    def solved(self):
        """Returns True if it has been solved, as in every single cell has
        only one option."""
        #print(numpy.all(self.allowed.sum(axis=2)==1))
        return numpy.all(self.allowed.sum(axis=2)==1) # 1 not 0
        
    
    def remove(self, row, col, value):
        """Marks a cell (row, col) to not allow the given value,
        i.e. sets its flag to be False."""
        # Mark the specified value as False...
        self.allowed[row, col, value-1] = False
        
        # If the cell is reduced to only have one value then remove that value
        # from all other cells with which it shares a peer group, by recursively
        # calling this method...
        if self.allowed[row, col, :].sum()==1:
        #returns index of all possible locations which dont have numbers init
            remain = numpy.nonzero(self.allowed[row, col, :])[0][0]
            
            # Same row peer group...
            #ammendment change ramge to 9
            for r in range(9):
                if r==row:
                    continue
            #Ammendment close bracket
                if self.allowed[r, col, remain]:
                    self.remove(r, col, remain+1)
            
            
            # Same col peer group...
            for c in range(9):
                if c==col:
                    continue
                if self.allowed[row, c, remain]:
                    self.remove(row, c, remain+1)
                    
            
            # Same square peer group (.indices converts a slice into the parameters of a range)...
            for r in range(*self.__square[row].indices(9)):
                for c in range(*self.__square[col].indices(9)):
                    #ammendment
                    if r==row and c==col:
                        continue
                    if self.allowed[r, c, remain]:
                        self.remove(r, c, remain+1)
        
        # Check if there are any trivial moves we can do, which in all cases
        # means there is a peer group where a value can only be placed in a
        # single cell, in which case fix that cell to be only that value.
        # Only need to check peer groups associated with the cell just edited,
        # and the specific value just removed...
        
        ## Same row peer group...
        target = self.allowed[row, :, value-1]
        if target.sum()==1:
            col_in = numpy.nonzero(target)[0][0]
            for val in range(1, 10):
                if val==value:
                    continue
                if self.allowed[row, col_in, val-1]:
                    self.remove(row, col_in, val)
        
        ## Same cell peer group...
        target = self.allowed[:, col, value-1]
        if target.sum()==1:
            row_in = numpy.nonzero(target)[0][0]
            for val in range(1, 10):
                if val==value:
                    continue
                if self.allowed[row_in, col, val-1]:
                    self.remove(row_in, col, val)

        ## Same square peer group...
        target = self.allowed[self.__square[row], self.__square[col], value-1]
        if target.sum()==1:
            row_in, col_in = [arr[0] for arr in numpy.nonzero(target)]
            row_in += self.__square[row].start
            col_in += self.__square[col].start
            
            for val in range(1, 10):
                if val==value:
                    continue
                if self.allowed[row_in, col_in, val-1]:
                    self.remove(row_in, col_in, val)
    
    
    def minimum(self):
        """Finds a cell that isn't yet unambiguous, i.e. has more than one choice of values
        and returns (row, column, options), where options is a numpy array of values it could
        take, e.g. [7, 9]. It always selects a cell with the least number of options, as
        this will typically speed up a search (probability of selecting the right one by
        random chance is higher with less options!). Will crash if .solved()==True."""
        # Count the number of options in each cell...
        counts = self.allowed.sum(axis=2)
        
        # Find coordinates of a cell that has the lowest count...
        lowest = numpy.min(counts[counts > 1])
        row, col = [axis[0] for axis in numpy.nonzero(counts == lowest)]
        
        # Get list of choices and return...
        
        #ammended to ZERO want to get inside list identifise choices for the cell with least number of choices
        choices = numpy.nonzero(self.allowed[row, col, :])[0]+1
        #print(choices)
        return row, col, choices
    
    
    def solve(self):
        """A generator, that yields every possible solution to this puzzle. Will
        not yield anything if there is no solution, and yield more than one if
        there are multiple solutions, noting that this means it is an invalid
        puzzle (A valid Sudoku only has one solution.). Do keep in mind that is
        a puzzle has lots of solutions this can go on for a very long time and
        consume all of the memory - you will always to put an upper limit on how
        many solutions to collect before breaking from the loop."""
        # If invalid better give up...
        if not self.valid():
            return
        
        # Base case...
        if self.solved():
            yield self
            return
            
        # Perform search...
        row, col, choices = self.minimum()
        
        for choice in choices:
            child = self.copy()
            
            for val in range(1, 10):
                if val==choice:
                    continue
                    
                if child.allowed[row, col, val-1]:
                    child.remove(row, col, val)
            
            for solution in child.solve():
                yield solution
    
    
    def random(self):
        """Returns a random cell under the condition that it still has at least two
        choices, with the exact return as minimum: (row, column, options)."""
        # Count the number of options in each cell...
        counts = self.allowed.sum(axis=2)
        
        # Select at random from those with a choice...
        row, col = numpy.nonzero(counts > 1)
        
        index = numpy.random.randint(row.shape[0])
        row = row[index]
        col = col[index]
        
        # Get choices and return...
        choices = numpy.nonzero(self.allowed[row, col, :])[0] + 1
        return row, col, choices


    def __repr__(self):
        """Returns a length 81 string that is the same as the representation
        used by the load() method."""
        ret = ['0'] * 81
        
        for row in range(9):
            for col in range(9):
                if self.allowed[row, col, :].sum()==1:
                    value = numpy.nonzero(self.allowed[row, col, :])[0][0] + 1
                    ret[row*9 + col] = str(value)
    
        return ''.join(ret)
    
    
    @staticmethod
    def prettyprint(s):
        """Pretty prints a Sudoku grid (returns a string). Its input is a string
        as described by the load method and returned by repr(SudokuState).
        Provided as a static method so you can print a puzzle that has not
        been converted into a SudokuState, to avoid the automatic constraint
        propagation that occurs when you do."""
        ret = []
        
        for row in range(9):
            line = []
            for col in range(9):
                t = s[row*9 + col]
                line.append(t if t!='0' else ' ')
            #print('line',line)
            str1 = str(' '.join(line[:3]))
            str2 = str(' '.join(line[3:6]))
            str3 = str(' '.join(line[6:9]))
            
            str4 = str1
            #ret.append('|'.join(line[:9]))
            retList = [' '.join(line[:3]),' '.join(line[3:6]),' '.join(line[6:9])]

            str1 = ''
            for x in range(3):
                str1 = str1 + ' ' +str(line[x])
                
            str2 = ''
            for x in range(3,6):
                #print(str(line[x]))
                str2 = str2 + ' ' + str(line[x])
            
            str3 = ''
            for x in range(6,9):
                #print(str(line[x]))
                str3 = str3 + ' ' + str(line[x])

            
            mylist = [str1,str2,str3]
            str4 = '|'.join(mylist)
            ret.append(str4)
            
            if row in [2, 5]:
                ret.append('-----+-----+-----')
        return '\n'.join(ret)


    def __str__(self):
        """Returns a nice text art representation of the grid."""
        return self.prettyprint(repr(self))


     
def generate():
    """Generates and returns a new Sudoku puzzle. Return is actually
    (string, solved, attempts), where string is the length 81 string
    decribed as part of SudokuState.load, and solved is the fully solved
    SudokuState object. attempts is how many puzzles it generated before
    one was valid."""
    # Use a 'while True' loop to keep trying until a generation works...
    attempts = 0
    success = 0
    while True:
        attempts += 1
        
        # Required state - the puzzle, the state to solve it and which digits have been used...
        puzzle = ['0'] * 81
        state = SudokuState()
        
        #print(state)
        # Fill in 17 cells, without violating any constraints...
        for i in range(17):
            row, col, choices = state.random()
            
            
            value = numpy.random.choice(choices)
            
            puzzle[row*9 + col] = str(value)
            
            for val in range(1,10):
                if val==value:
                    continue
                if state.allowed[row, col, val-1]:
                    state.remove(row, col, val)

        # Solve, collecting upto two solutions (no point collecting more),
        # adding a digit each time it gets more than one solution, until it
        # has one solution or has failed...
        while True:
            solutions = []
            for solution in state.solve():
                solutions.append(solution)
                if len(solutions)>1:
                    break

            if len(solutions)<2:
                break
            
            row, col, choices = state.random()
            value = numpy.random.choice(choices)
            
            puzzle[row*9 + col] = str(value)
            
            for val in range(1,10):
                if val==value:
                    continue
                if state.allowed[row, col, val-1]:
                    state.remove(row, col, val)

        # Return if we have exactly one solution...
        if len(solutions)==1:
            # It's valid! - return...
            success = 1
            return ''.join(puzzle), solutions[0], attempts


## Generate

For convenience the below code will generate a Sudoku and print its starting state and solution.

In [5]:
puzzle, solution, attempts = generate()
print('Took {} random puzzles before success'.format(attempts))
print()

print('Puzzle:')
print(puzzle)
print(SudokuState.prettyprint(puzzle))
print()

print('Solution:')
print(solution)

Took 2 random puzzles before success

Puzzle:
007900000090300006564208307003020000200800000008400059602050800100000003000080500
     7| 9    |      
   9  | 3    |     6
 5 6 4| 2   8| 3   7
-----+-----+-----
     3|   2  |      
 2    | 8    |      
     8| 4    |   5 9
-----+-----+-----
 6   2|   5  | 8    
 1    |      |     3
      |   8  | 5    

Solution:
 3 2 7| 9 6 4| 1 8 5
 8 9 1| 3 7 5| 4 2 6
 5 6 4| 2 1 8| 3 9 7
-----+-----+-----
 9 4 3| 5 2 7| 6 1 8
 2 5 6| 8 9 1| 7 3 4
 7 1 8| 4 3 6| 2 5 9
-----+-----+-----
 6 3 2| 7 5 9| 8 4 1
 1 8 5| 6 4 2| 9 7 3
 4 7 9| 1 8 3| 5 6 2


The file `examples.txt` provides valid Sudoku puzzles encoded using the representation described in the `load()` method of the `SudokuState` class above.

In [6]:
f = open("examples.txt", "r")

# extract puzzles
state = []
for row in f:
    state.append(row)
    
# solve puzzles
for puzzle in state:
    
    puzzle, solution, attempts = generate()
    
    print('Puzzle:')
    print(SudokuState.prettyprint(puzzle))
    print()

    print('Solution:')
    print(solution)
    
    

Puzzle:
      | 5   1|     4
     9|      |     2
      | 7    | 5 8  
-----+-----+-----
 2 9  | 6    |      
 7   1|   5  |   6  
   5 3| 9    |     7
-----+-----+-----
      |   9 2|      
     7| 4    |     8
      |     8| 4   6

Solution:
 8 7 2| 5 3 1| 6 9 4
 5 3 9| 8 4 6| 1 7 2
 1 6 4| 7 2 9| 5 8 3
-----+-----+-----
 2 9 8| 6 1 7| 3 4 5
 7 4 1| 2 5 3| 8 6 9
 6 5 3| 9 8 4| 2 1 7
-----+-----+-----
 4 8 6| 3 9 2| 7 5 1
 3 1 7| 4 6 5| 9 2 8
 9 2 5| 1 7 8| 4 3 6
Puzzle:
   3 1|   7 5|     9
     6|      |     1
     5|     8|   7  
-----+-----+-----
      | 4   2| 6    
   9  |   6  |   2  
 6    | 3 9  |     5
-----+-----+-----
 8    | 7    |      
     3|      | 7 4  
   7  |   5  |     6

Solution:
 4 3 1| 2 7 5| 8 6 9
 7 8 6| 9 3 4| 2 5 1
 9 2 5| 6 1 8| 3 7 4
-----+-----+-----
 1 5 7| 4 8 2| 6 9 3
 3 9 8| 5 6 1| 4 2 7
 6 4 2| 3 9 7| 1 8 5
-----+-----+-----
 8 1 9| 7 4 6| 5 3 2
 5 6 3| 1 2 9| 7 4 8
 2 7 4| 8 5 3| 9 1 6
Puzzle:
 5    | 7   9| 3    
   3 6|     1|     5
   2 1|     

Puzzle:
   2 4| 8 3  |   1  
     6|      | 2    
 9 1  | 6    |     7
-----+-----+-----
   9  | 3   8|   5 1
 4    | 9    |     6
     5| 1 2  | 8 4  
-----+-----+-----
 5    |      |     8
     8| 5   7|      
      |      |      

Solution:
 7 2 4| 8 3 9| 6 1 5
 8 5 6| 7 1 4| 2 9 3
 9 1 3| 6 5 2| 4 8 7
-----+-----+-----
 6 9 2| 3 4 8| 7 5 1
 4 8 1| 9 7 5| 3 2 6
 3 7 5| 1 2 6| 8 4 9
-----+-----+-----
 5 4 7| 2 6 1| 9 3 8
 2 3 8| 5 9 7| 1 6 4
 1 6 9| 4 8 3| 5 7 2
Puzzle:
 9    | 4    | 5 7  
      |      |      
   6  |   9 7|   3  
-----+-----+-----
   5 1|      |     7
      | 3    | 8 1 2
 4   8|      | 3   6
-----+-----+-----
 6   4|      |     5
      |   4  | 1 2  
   3 2|   7  |     8

Solution:
 9 8 3| 4 6 2| 5 7 1
 2 4 7| 1 3 5| 6 8 9
 1 6 5| 8 9 7| 2 3 4
-----+-----+-----
 3 5 1| 6 2 8| 9 4 7
 7 9 6| 3 5 4| 8 1 2
 4 2 8| 7 1 9| 3 5 6
-----+-----+-----
 6 1 4| 2 8 3| 7 9 5
 8 7 9| 5 4 6| 1 2 3
 5 3 2| 9 7 1| 4 6 8
Puzzle:
 3 7  | 4    |      
   6 9| 8   5|     4
 1    | 2   

Puzzle:
      | 5    |     1
      | 6    | 4    
      |   1  |      
-----+-----+-----
     9|      | 3    
 8   6|     4| 9    
 1    | 8 2 9|     7
-----+-----+-----
     4|      |   2  
   7  | 3   6|   9 5
      |     1| 6   8

Solution:
 4 6 2| 5 9 7| 8 3 1
 5 3 1| 6 8 2| 4 7 9
 9 8 7| 4 1 3| 2 5 6
-----+-----+-----
 7 2 9| 1 6 5| 3 8 4
 8 5 6| 7 3 4| 9 1 2
 1 4 3| 8 2 9| 5 6 7
-----+-----+-----
 6 1 4| 9 5 8| 7 2 3
 2 7 8| 3 4 6| 1 9 5
 3 9 5| 2 7 1| 6 4 8
Puzzle:
   1 5| 8    |   7  
 2    |      |     1
 6 9  | 5   1|      
-----+-----+-----
 4 7  | 6 1  |   2 5
   6  |      |   1  
 5    | 4   7| 6 9  
-----+-----+-----
      |   7 4| 9    
      |     3|      
   5  |      |   4  

Solution:
 3 1 5| 8 6 2| 4 7 9
 2 4 8| 7 3 9| 5 6 1
 6 9 7| 5 4 1| 8 3 2
-----+-----+-----
 4 7 9| 6 1 8| 3 2 5
 8 6 2| 3 9 5| 7 1 4
 5 3 1| 4 2 7| 6 9 8
-----+-----+-----
 1 8 6| 2 7 4| 9 5 3
 7 2 4| 9 5 3| 1 8 6
 9 5 3| 1 8 6| 2 4 7
Puzzle:
   7  |     3|      
     6| 1    |   3  
     2| 5   

Puzzle:
      |   3  |     4
 7    | 6 4  |   3  
      |      | 9 6  
-----+-----+-----
 8   5| 4 1  |     9
     9| 7    |   8 1
   3 7|      |   4  
-----+-----+-----
   9 1|     5|      
   8 3|      | 6 1  
 5   4|      | 2    

Solution:
 9 1 6| 2 3 7| 8 5 4
 7 5 8| 6 4 9| 1 3 2
 3 4 2| 5 8 1| 9 6 7
-----+-----+-----
 8 6 5| 4 1 3| 7 2 9
 4 2 9| 7 5 6| 3 8 1
 1 3 7| 8 9 2| 5 4 6
-----+-----+-----
 6 9 1| 3 2 5| 4 7 8
 2 8 3| 9 7 4| 6 1 5
 5 7 4| 1 6 8| 2 9 3
Puzzle:
 7 8 9| 6    |      
 6 1  |   8  |      
      |     3| 6 8  
-----+-----+-----
     5| 9 7 6| 8   3
   4  |      |   9  
      |   4 1| 5    
-----+-----+-----
 5 3  | 7   4|      
 4   6| 8   2|      
   7  |   3  |      

Solution:
 7 8 9| 6 2 5| 3 1 4
 6 1 3| 4 8 7| 2 5 9
 2 5 4| 1 9 3| 6 8 7
-----+-----+-----
 1 2 5| 9 7 6| 8 4 3
 3 4 7| 2 5 8| 1 9 6
 9 6 8| 3 4 1| 5 7 2
-----+-----+-----
 5 3 1| 7 6 4| 9 2 8
 4 9 6| 8 1 2| 7 3 5
 8 7 2| 5 3 9| 4 6 1
Puzzle:
   3  |      |   6 1
   5 4|      | 7    
 1 6  | 8 5 

Puzzle:
 7    |      | 4 6  
     9| 4 3 6| 2 8  
      |     8| 1    
-----+-----+-----
 3 7  | 1 2 5| 8    
      | 8    |      
 2   5|      |   7  
-----+-----+-----
      |   4 1|   2  
 1   3| 5 7  |      
      |      |      

Solution:
 7 3 8| 2 1 9| 4 6 5
 5 1 9| 4 3 6| 2 8 7
 6 2 4| 7 5 8| 1 3 9
-----+-----+-----
 3 7 6| 1 2 5| 8 9 4
 9 4 1| 8 6 7| 3 5 2
 2 8 5| 3 9 4| 6 7 1
-----+-----+-----
 8 9 7| 6 4 1| 5 2 3
 1 6 3| 5 7 2| 9 4 8
 4 5 2| 9 8 3| 7 1 6
Puzzle:
   8 5| 9    |      
     9|      | 6    
 4    |   5  | 2 3  
-----+-----+-----
      |   8  | 4    
 6   2|      | 7 5  
      |     2| 3 9  
-----+-----+-----
 9   6| 4   8|   2  
   1 7|   3  |      
   2 4|     1| 9    

Solution:
 3 8 5| 9 2 6| 1 4 7
 2 7 9| 3 1 4| 6 8 5
 4 6 1| 8 5 7| 2 3 9
-----+-----+-----
 7 9 3| 6 8 5| 4 1 2
 6 4 2| 1 9 3| 7 5 8
 1 5 8| 7 4 2| 3 9 6
-----+-----+-----
 9 3 6| 4 7 8| 5 2 1
 5 1 7| 2 3 9| 8 6 4
 8 2 4| 5 6 1| 9 7 3
Puzzle:
      |      |   4  
     9|   8 6| 5 2 3
     3| 1   

Puzzle:
   8  |   1 4|      
   2  |     3| 9    
   1 5| 2    |      
-----+-----+-----
      |      | 8    
   4 9| 3   7| 1    
     1|   9 5|      
-----+-----+-----
   7  |     6|     4
      |   5 2|      
   5 4|     9| 6 1  

Solution:
 6 8 3| 9 1 4| 2 5 7
 4 2 7| 5 6 3| 9 8 1
 9 1 5| 2 7 8| 3 4 6
-----+-----+-----
 5 3 2| 6 4 1| 8 7 9
 8 4 9| 3 2 7| 1 6 5
 7 6 1| 8 9 5| 4 2 3
-----+-----+-----
 2 7 8| 1 3 6| 5 9 4
 1 9 6| 4 5 2| 7 3 8
 3 5 4| 7 8 9| 6 1 2
Puzzle:
      |     1|   8 4
   3 9|   8  | 2    
   1  | 5 2  | 7 9  
-----+-----+-----
      |      |      
   2  | 6   7|     5
     7|      | 8    
-----+-----+-----
     2|     9|      
     3|      |   4  
     4|     8|   5 6

Solution:
 2 7 6| 9 3 1| 5 8 4
 5 3 9| 7 8 4| 2 6 1
 4 1 8| 5 2 6| 7 9 3
-----+-----+-----
 9 4 5| 8 1 3| 6 2 7
 8 2 1| 6 9 7| 4 3 5
 3 6 7| 4 5 2| 8 1 9
-----+-----+-----
 6 5 2| 3 4 9| 1 7 8
 7 8 3| 1 6 5| 9 4 2
 1 9 4| 2 7 8| 3 5 6
Puzzle:
      |      |   6  
   8  | 7    | 2   3
      | 6   

Puzzle:
 3    |     6| 2   4
 5    |      | 7    
 4   8|   9  | 6    
-----+-----+-----
     3|   5  |     2
     6|   1 9| 4    
 9    |      |   1 7
-----+-----+-----
      | 8    | 1    
 1    |      | 3 4  
 6   4| 5 7  | 8    

Solution:
 3 7 9| 1 8 6| 2 5 4
 5 6 1| 3 2 4| 7 9 8
 4 2 8| 7 9 5| 6 3 1
-----+-----+-----
 8 1 3| 4 5 7| 9 6 2
 7 5 6| 2 1 9| 4 8 3
 9 4 2| 6 3 8| 5 1 7
-----+-----+-----
 2 9 5| 8 4 3| 1 7 6
 1 8 7| 9 6 2| 3 4 5
 6 3 4| 5 7 1| 8 2 9
Puzzle:
   1 3|      |   9 7
      | 9    | 2    
 9    |   4 7|     1
-----+-----+-----
      |      |      
   7 9|   3 1|      
 2   1|   5 4|      
-----+-----+-----
 6 5  |      |     9
   2  |   9  | 7 5 8
      | 2 7  |     3

Solution:
 4 1 3| 5 6 2| 8 9 7
 7 6 5| 9 1 8| 2 3 4
 9 8 2| 3 4 7| 5 6 1
-----+-----+-----
 8 4 6| 7 2 9| 3 1 5
 5 7 9| 6 3 1| 4 8 2
 2 3 1| 8 5 4| 9 7 6
-----+-----+-----
 6 5 7| 4 8 3| 1 2 9
 3 2 4| 1 9 6| 7 5 8
 1 9 8| 2 7 5| 6 4 3
Puzzle:
 4    |      | 1 7  
 6 1  |      |     2
   3 2| 4 7 

Puzzle:
     9| 5    |   7  
      | 2   7|   1 6
   4  |      |     5
-----+-----+-----
 2 5 7|   3  |   9  
      | 8    |      
 8 3  |   4  |      
-----+-----+-----
      | 9    | 5 2  
   2  | 6    | 1    
      |   2 8|   6 3

Solution:
 1 6 9| 5 8 4| 3 7 2
 5 8 3| 2 9 7| 4 1 6
 7 4 2| 3 6 1| 9 8 5
-----+-----+-----
 2 5 7| 1 3 6| 8 9 4
 4 9 1| 8 5 2| 6 3 7
 8 3 6| 7 4 9| 2 5 1
-----+-----+-----
 6 7 4| 9 1 3| 5 2 8
 3 2 8| 6 7 5| 1 4 9
 9 1 5| 4 2 8| 7 6 3
Puzzle:
      |     1| 9    
 8 9  | 7   2|   6  
 6 2  |      | 1   7
-----+-----+-----
 1    |     4|     3
   8  | 1    | 6    
   4  |      |     2
-----+-----+-----
 4    |      |      
      | 9    | 3 7  
 3    |   6  | 8   4

Solution:
 7 3 4| 6 5 1| 9 2 8
 8 9 1| 7 3 2| 4 6 5
 6 2 5| 4 8 9| 1 3 7
-----+-----+-----
 1 6 2| 5 9 4| 7 8 3
 5 8 7| 1 2 3| 6 4 9
 9 4 3| 8 7 6| 5 1 2
-----+-----+-----
 4 7 8| 3 1 5| 2 9 6
 2 5 6| 9 4 8| 3 7 1
 3 1 9| 2 6 7| 8 5 4
Puzzle:
   9 7|   5 3|   8  
     4| 7    |   9  
     8|     

Puzzle:
      |   8  | 5 2  
   7 6| 4 2 9| 3 1  
     2|   1  |      
-----+-----+-----
 7 5  | 2    |      
   4 9|     8| 2    
   3 1| 7 9  |   6  
-----+-----+-----
      |      |      
 8   7| 5 3  | 6    
     3|   4 2|   5 9

Solution:
 9 1 4| 3 8 7| 5 2 6
 5 7 6| 4 2 9| 3 1 8
 3 8 2| 6 1 5| 9 7 4
-----+-----+-----
 7 5 8| 2 6 3| 4 9 1
 6 4 9| 1 5 8| 2 3 7
 2 3 1| 7 9 4| 8 6 5
-----+-----+-----
 4 2 5| 9 7 6| 1 8 3
 8 9 7| 5 3 1| 6 4 2
 1 6 3| 8 4 2| 7 5 9
Puzzle:
      |   3  |     4
 8   1|      |     6
 5    | 8   7| 2   3
-----+-----+-----
 3   2|      | 1    
   8  |     6| 5   2
 9    | 1    |   4  
-----+-----+-----
   5  |   9  | 6    
 6   7|      |      
      |   5 4| 7 8 9

Solution:
 7 2 6| 9 3 1| 8 5 4
 8 3 1| 4 2 5| 9 7 6
 5 4 9| 8 6 7| 2 1 3
-----+-----+-----
 3 7 2| 5 4 9| 1 6 8
 1 8 4| 3 7 6| 5 9 2
 9 6 5| 1 8 2| 3 4 7
-----+-----+-----
 4 5 8| 7 9 3| 6 2 1
 6 9 7| 2 1 8| 4 3 5
 2 1 3| 6 5 4| 7 8 9
